In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
with open('content3.html','r', encoding='utf-8') as f:
    response = f.read()
soup = BeautifulSoup(response, 'html.parser')


In [13]:
html_fragment = soup.find_all(class_='row_card')

In [29]:
html_fragment[1].find( class_='right').get_text(strip=True).replace('\n','').replace(' ','')

'02-2055/2016∼М-55672/2016'

In [16]:
html_fragment[1]

<div class="row_card">
<div class="left">                Номер дела ~ материала
    </div>
<div class="right">
                    02-2055/2016

            
                            ∼                                     М-55672/2016                                    
    </div>
</div>

In [131]:
dict_values = {
    'Уникальный идентификатор дела':None,
    'Номер дела ~ материала': None,
    'Номер жалобы': None,
    'Истец':None,
    "Ответчик": None,
    'Заявитель':None,
    'Cудья': None,
    'Статья КоАП РФ': None,
    'Суд, вынесший решение':None,
    'Категория дела': None,
    'Текущее состояние':None,
    'Результат рассмотрения':None

}

In [132]:
html_fragment = soup.find_all(class_='row_card')

for html in html_fragment:

    key = html.find(class_='left').get_text(strip=True)
    
    if key == 'Номер дела ~ материала':
        key = key.replace('\n','').replace(' ','')
        
    if key in dict_values.keys():
        value = html.find(class_='right').get_text(strip=True)
        dict_values[key] = value
        
    if key == 'Стороны':
        strong_tags = html.find_all('strong')
        for strong_tag in strong_tags:
            text = strong_tag.get_text(strip=True)
            next_sibling = strong_tag.next_sibling
            if 'Истец' in text[:5]:
                dict_values['Истец'] = next_sibling.strip() if next_sibling else None
            if 'Ответ' in text[:5]:
                dict_values['Ответчик'] = next_sibling.strip() if next_sibling else None
            if 'Заявител' in text[:8]:
                dict_values['Заявитель'] = next_sibling.strip() if next_sibling else None


dict_values

{'Уникальный идентификатор дела': '77RS0020-01-2016-008801-05',
 'Номер дела ~ материала': None,
 'Номер жалобы': None,
 'Истец': None,
 'Ответчик': None,
 'Заявитель': '"Сбербанк России" в лице филиала Московского банка ОАО "Сбербанк России"',
 'Cудья': 'Примак В.Г.',
 'Категория дела': '237 - Иные дела особого производства',
 'Текущее состояние': 'Удовлетворено, 30.03.2016',
 'Результат рассмотрения': None,
 'Суд, вынесший решение': None}

In [40]:
def get_info(html_fragment):
    dict_values = {
    'id': None,
    'Уникальный идентификатор дела':None,
    'Номер дела ~ материала': None,
    'Номер жалобы': None,
    'Истец':None,
    "Ответчик": None,
    'Заявитель':None,
    'Cудья': None,
    'Статья КоАП РФ': None,
    'Суд, вынесший решение':None,
    'Категория дела': None,
    'Текущее состояние':None,
    'Результат рассмотрения':None,
    'path':None}

    html_fragment = soup.find_all(class_='row_card') # ищем всю нужную информацию 

    for html in html_fragment:

        key = html.find(class_='left').get_text(strip=True)   
            
        if key in dict_values.keys():
            value = html.find(class_='right').get_text(strip=True)
            dict_values[key] = value
        
        if key == 'Номер дела ~ материала' or key == 'Номер дела':
            value_ = html.find(class_='right').get_text(strip=True).replace('\n','').replace(' ','')
            dict_values['Номер дела ~ материала'] = value_
            
        if key == 'Стороны': # обработка поля Стороны
            strong_tags = html.find_all('strong')
            for strong_tag in strong_tags:
                text = strong_tag.get_text(strip=True)
                next_sibling = strong_tag.next_sibling
                if 'Истец' in text[:5]:
                    dict_values['Истец'] = next_sibling.strip() if next_sibling else None
                if 'Ответ' in text[:5]:
                    dict_values['Ответчик'] = next_sibling.strip() if next_sibling else None
                if 'Заявител' in text[:8]:
                    dict_values['Заявитель'] = next_sibling.strip() if next_sibling else None

    return dict_values
get_info(soup)

{'id': None,
 'Уникальный идентификатор дела': '77RS0017-01-2016-016888-56',
 'Номер дела ~ материала': '02-2055/2016∼М-55672/2016',
 'Номер жалобы': None,
 'Истец': 'Сбербанк России',
 'Ответчик': 'Касаримова А. Н.',
 'Заявитель': None,
 'Cудья': 'Клинцова И.В.',
 'Статья КоАП РФ': None,
 'Суд, вынесший решение': None,
 'Категория дела': '213 - О взыскании сумм по договору займа, кредитному договору',
 'Текущее состояние': 'Удовлетворено, 02.03.2016',
 'Результат рассмотрения': None,
 'path': None}

In [12]:
info = get_info(soup)
[list(info.values())]

[['77RS0017-01-2016-016888-56',
  None,
  None,
  'Сбербанк России',
  'Касаримова А. Н.',
  None,
  'Клинцова И.В.',
  None,
  None,
  '213 - О взыскании сумм по договору займа, кредитному договору',
  'Удовлетворено, 02.03.2016',
  None]]

In [53]:
suggested_filename = 'abd'
save_directory = rf'C:\Data\Visual Studio\Kursova_2\documents\{suggested_filename}'
path_for_bd = save_directory.split('\\')[-2:]
path_for_bd = '\\'.join(path_for_bd)
path_for_bd

SyntaxError: EOL while scanning string literal (1812394387.py, line 4)

In [ ]:
# Уникальный_идентификатор_дела varchar(50) DEFAULT NULL,
#             Номер_дела varchar(50) DEFAULT NULL,
#             Истец varchar(200) DEFAULT NULL,
#             Ответчик varchar(200) DEFAULT NULL,
#             Судья varchar(20) DEFAULT NULL,
#             Статья_КоАП_РФ varchar(50) DEFAULT NULL,
#             Категория_дела varchar(100) DEFAULT NULL,
#             Текущее_состояние varchar(100) DEFAULT NULL,
#             Результат_рассмотрения varchar(100) DEFAULT NULL,
#             Ссылка_на_документ varchar(50) DEFAULT NULL
#             Номер жалобы
#               Суд, вынесший решение

In [93]:
from bs4 import BeautifulSoup

html_fragment = '''
<div class="right"><strong>Истец: </strong>Сбербанк России<br/><strong>Ответчик: </strong>Касаримова А. Н.<br/></div>
'''

soup = BeautifulSoup(html_fragment, 'html.parser')

# Функция для извлечения информации об истце и ответчике
def extract_party_info(html_content):
    # Парсинг HTML-фрагмента
    soup = BeautifulSoup(html_content, 'html.parser')
    right_div = soup.find('div', class_='right')
    if not right_div:
        return "Информация не найдена."
    
    result = {}
    
    # Ищем все теги <strong> и анализируем их содержимое
    strong_tags = right_div.find_all('strong')
    for strong_tag in strong_tags:
        text = strong_tag.get_text(strip=True)
        next_sibling = strong_tag.next_sibling
        
        if 'Истец:' in text:
            result['Истец'] = next_sibling.strip() if next_sibling else 'не указан'
        elif 'Ответчик:' in text:
            result['Ответчик'] = next_sibling.strip() if next_sibling else 'не указан'
    
    return result

info = extract_party_info(html_fragment)

for key, value in info.items():
    print(f"{key}: {value}")


Истец: Сбербанк России
Ответчик: Касаримова А. Н.


In [95]:
soup = BeautifulSoup(html_fragment, 'html.parser')
right_div = soup.find('div', class_='right')
if not right_div:
    print("Информация не найдена.")

result = {}

# Ищем все теги <strong> и анализируем их содержимое
strong_tags = right_div.find_all('strong')
for strong_tag in strong_tags:
    text = strong_tag.get_text(strip=True)
    next_sibling = strong_tag.next_sibling
    
    if 'Истец' in text[:5]:
        result['Истец'] = next_sibling.strip() if next_sibling else 'не указан'
    elif 'Ответ' in text[:5]:
        result['Ответчик'] = next_sibling.strip() if next_sibling else 'не указан'

In [99]:
strong_tags[0].get_text(strip=True)

'Истец:'

In [100]:
strong_tags[0].next_sibling

'Сбербанк России'

In [110]:
'Ответ:' in text

False

In [116]:
'Истец:'[:5]


'Истец'

In [113]:
"Ответчик"[:5]

'Ответ'